In [1]:
!python -V

Python 3.9.7


In [2]:
import numpy as np

In [10]:
import tensorflow as tf
import tensorflow.keras as keras

In [6]:
tf.__version__

'2.11.0'

In [8]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.xception import preprocess_input

# Convert model to tflite

In [11]:
model = keras.models.load_model('dino_dragon_10_0.899.h5')

In [15]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with open('dino_dragon.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\admin\AppData\Local\Temp\tmpcqhvou9w\assets


INFO:tensorflow:Assets written to: C:\Users\admin\AppData\Local\Temp\tmpcqhvou9w\assets


# Input and output indexes

In [14]:
import tensorflow.lite as tflite

In [16]:
interpreter = tflite.Interpreter(model_path='dino_dragon.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

In [17]:
print(f"input={input_index} output={output_index}")

input=0 output=13


# Preprocess images

In [18]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img


In [30]:
IMG_URL = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/df/Smaug_par_David_Demaret.jpg/1280px-Smaug_par_David_Demaret.jpg"
IMG_SIZE = (150, 150)
img = download_image(IMG_URL)
img = prepare_image(img, IMG_SIZE)

In [43]:
def preprocess_image(img):
    X = np.array(img) / 255
    return [X.astype(np.float32)]

In [44]:
X = preprocess_image(img)

In [46]:
X[0][0,0,0]

0.5529412

# Evaluate the model

In [47]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

In [48]:
preds

array([[0.82448614]], dtype=float32)